In [1]:
import numpy as np
import struct
import sys

 The form '!' is available for those poor souls who claim they can’t remember whether network byte order is big-endian or little-endian.
 https://docs.python.org/2.7/library/struct.html#byte-order-size-and-alignment
 
 モトローラー系とインテル系のバイトオーダーが逆

In [2]:
# システムのバイトオーダーを表示
print sys.byteorder

little


In [3]:
def string_byte(string_data):
    if isinstance(string_data, unicode):
        print 'unicode'
    elif isinstance(string_data, str):
        print 'str'
    
    orded = ord(string_data)
    print "文字   ", string_data
    print "10進数 ", orded
    print "16進数  0x{0:X}".format(orded)
    if isinstance(string_data, unicode):
        print "逆変換 ", unichr(orded)
    elif isinstance(string_data, str):
        print "逆変換 ", chr(orded)
    print

len は、文字列の場合はバイト数、Unicode文字列の場合は文字数が取得できる

In [23]:
print type("あ") # string型
print type(u"あ") # unicode型

<type 'str'>
<type 'unicode'>


In [4]:
print len("あ") # バイト数
print len(u"あ") # 文字列の長さ

3
1


In [24]:
string_byte("A")
string_byte(u"あ")
#string_byte("あ") # ordはunicode

str
文字    A
10進数  65
16進数  0x41
逆変換  A

unicode
文字    あ
10進数  12354
16進数  0x3042
逆変換  あ



| フォーマット | 型(C)              | 型の説明       | Standard size |
|--------------|--------------------|----------------|----------|
| x            | pad byte           |                |          |
| c            | char               | 長さ１の文字列 | 1        |
| b            | signed char        | 整数型         | 1        |
| B            | unsigned char      | 整数型         | 1        |
| ?            | \_Bool             | 真偽値型       | 1        |
| h            | short              | 整数型         | 1        |
| H            | unsigned short     | 整数型         | 2        |
| i            | int                | 整数型         | 4        |
| I            | unsigned int       | 整数型         | 4        |
| l            | long               | 整数型         | 4        |
| L            | unsigned long      | 整数型         | 4        |
| q            | long long          | 整数型         | 8        |
| Q            | unsigned long long | 整数型         | 8        |
| f            | float              | 浮動小数点型   | 4        |
| d            | double             | 浮動小数点型   | 8        |
| s            | char[]             | 文字列         |          |
| p            | char[]             | 文字列         |          |
| P            | void\*             | 整数型         |          |

| 文字 | バイトオーダ                        | サイズ   | アラインメント |
|------|-------------------------------------|----------|----------------|
| @    | native                              | native   | native         |
| =    | native                              | standard | none           |
| <	   | リトルエンディアン                  | standard | none           |
| >	   | ビッグエンディアン                  | standard | none           |
| !    | ネットワーク (= ビッグエンディアン) | standard | none           |

In [6]:
#ビッグエンディアンで文字列「あいうえお」を表示
struct.pack("<5s", "あいうえお")

'\xe3\x81\x82\xe3\x81'

In [7]:
#リトルエンディアンで文字列「あいうえお」を表示
struct.pack(">5s", "あいうえお")

'\xe3\x81\x82\xe3\x81'

In [8]:
#ビッグエンディアンで3.14を表示
struct.pack(">f", 3.14)

'@H\xf5\xc3'

In [9]:
#リトルエンディアンで3.14を表示
struct.pack("<f", 3.14)

'\xc3\xf5H@'

In [10]:
#2つの数字を並べて表示
struct.pack(">2f", 3.14, 1592)

'@H\xf5\xc3D\xc7\x00\x00'

https://ja.wikipedia.org/wiki/IEEE_754

IEEE 754（IEEE 浮動小数点数演算標準）

<img src="./ieee754_double.png">

$(−1)^s × c × b^q$  
s : 符号（0または1）  
c : 仮数  
q : 指数  
b : 基数 (2または10)  

### データをdouble型としてバイナリ化する。

In [11]:
data = -3.14
binarizedAsDouble = struct.pack(">d", data)
binarizedAsDouble

'\xc0\t\x1e\xb8Q\xeb\x85\x1f'

### バイト列をunsigned long long型として解釈しなおす。   
double型もunsigned long long型も64bitなので、ちょうど一つの値に解釈しなおすことができる。

In [12]:
unpackedAsULL = struct.unpack('>Q', binarizedAsDouble)[0]
unpackedAsULL

13837625107069764895L

### unsigned long long型で解釈し直したデータをバイナリデータとして見てみる。  
接頭辞の0bは二進数であることを示している。  
接頭辞の分だけ長くなっているので、lenで長さを測ると64+2=66bitになっている。  
\* 正の数をバイナリ化すると符号の部分が0になるため、lenで長さを測ると符号部が省略された63bitが返されてしまう。

In [13]:
print bin(unpackedAsULL)
print len(bin(unpackedAsULL)), 'bit'

0b1100000000001001000111101011100001010001111010111000010100011111
66 bit


IEEE754規格では、右から64bit目が符号を表している。  
ビット演算を行い、`unpackedAsULL`を63ビット右にシフトする。こうすることで、`unpackedAsULL`の右側63bit分の情報が消えて、最上位の64bit目の情報だけが取れる。

In [14]:
sign = (unpackedAsULL >> 63)
sign

1L

IEEE754規格では、右から53〜63bit目が指数を表している。  
指数はバイアスの`1023`が乗っかっているので、実際は`exp-1023`が指数である。

In [15]:
bin((1<<11)-1)  # 11 bit分のTrue。右から12bit目はFalse。
bin(unpackedAsULL >> 52) # 左から12 bitの情報
exp = (unpackedAsULL >> 52) & ((1<<11)-1) # 左から12 bitの情報を取り出し、さらに符号を表す一番左のbit情報を落す
print exp
print bin(exp)

1024
0b10000000000


IEEE754規格では、1〜52bit目が仮数を表している。

In [16]:
frac = unpackedAsULL & ((1<<52)-1)
print frac
print "{:}".format(bin(frac))
print bin(frac)[2:].zfill(53)
print bin(1<<52)[2:]
print bin(frac | 1<<52)

2567051787601183
0b1001000111101011100001010001111010111000010100011111
01001000111101011100001010001111010111000010100011111
10000000000000000000000000000000000000000000000000000
0b11001000111101011100001010001111010111000010100011111


double型の仮数は52bitなので、53bitでしか表現できない数の最小値（`1<<52`）で正規化されている。  
つまり、`1<<52 = 4,503,599,627,370,496`倍の値が格納されている。

In [17]:
print bin(1<<52), len(bin(1<<52)[2:]), 'bit'
print "{:.0f}".format(1<<52)
frac2 = frac/float(1<<52)
print frac2

0b10000000000000000000000000000000000000000000000000000 53 bit
4503599627370496
0.57


https://ja.wikipedia.org/wiki/倍精度浮動小数点数  
倍精度浮動小数点数は  
$\displaystyle{ (-1)^{sign} \Bigg( 1 + \sum^{52}_{i=1} b_{-i} 2^{-i} \Bigg) \times 2^{exp-1023} }$

In [18]:
(-1)**sign * (1+frac2) * (2 ** (exp-1023))

-3.14

# 16進数表記の半精度バイナリデータの変換

<img src="./ieee754_half.png">

NumPyの半精度浮動小数点数 0.5625 が、npy ファイルでは 80 38 と記録されている。  
リトルエンディアンであることに注意して、バイト列 80 38 から浮動小数点数に復号化する。  
リトルエンディアンなので読み込み順は後ろからとなり、16進数表記は 0x 38 80 であることに注意。

In [19]:
class binary16(object):
    nbit_tot = 15
    nbit_exp = 5
    nbit_frac = 10
    exp_bias = 15

class binary64(object):
    nbit_tot = 64
    nbit_exp = 11
    nbit_frac = 52
    exp_bias = 1023
    
def code2number(code, floatFormat):
    sign = code>>floatFormat.nbit_tot
    exp = (code>>floatFormat.nbit_frac) & ((1<<floatFormat.nbit_exp)-1)
    frac = code & ((1<<floatFormat.nbit_frac)-1)
    frac2 = frac/float(1<<floatFormat.nbit_frac)
    return (-1)**sign * (1+frac2) * (2 ** (exp-floatFormat.exp_bias))

In [20]:
# 16進数で3880は、半精度実数で0.5625に相当
print code2number(0x3880, binary16())

0.5625


In [21]:
# 16進数で3880は、倍精度実数で1.11253692926e-308に相当
print code2number(0x3880, binary64())

1.11253692926e-308
